In [ ]:
from PyPDF2 import PdfReader
import sys
import os

In [ ]:
print("choose a pdf file to read")


#file = input()

file ="48 Laws of Power, The - Robert Greene.pdf"


In [ ]:
## save extracted text to just one file seperated page by page

def save_pdf(file):
    pdf = PdfReader(file)
    with open("extracted_text.txt", "w") as f:
        for i in range(21,530):
            f.write(pdf.pages[i].extract_text())
            f.write("\n\n")

save_pdf(file)

In [ ]:
import os

# Define the path where the files will be saved
path = "Laws/"

# Create the directory if it does not exist
if not os.path.exists(path):
    os.makedirs(path)

# Read the content of the uploaded file
with open("extracted_text.txt", "r") as file:
    lines = file.readlines()

# Initialize variables to store the current law section and the law number
law_number = 1
current_law = []



In [ ]:

# Process each line to separate the sections
for line in lines:
    if line.startswith("LAW"):
        if current_law:
            # Save the current law section to a file
            with open(f"{path}law_{law_number}.txt", "w") as law_file:
                law_file.writelines(current_law)
            # Reset the current law section and increment the law number
            current_law = []
            law_number += 1
    # Append the line to the current law section
    current_law.append(line)


In [ ]:

# Save the last law section to a file if it exists
if current_law:
    with open(f"{path}law_{law_number}.txt", "w") as law_file:
        law_file.writelines(current_law)

In [ ]:
# şimdi her bir yasayı özetleyelim ve bir dosyaya yazalım

import re

# Define the path where the files will be saved

path = "Summaries/"

# Create the directory if it does not exist
if not os.path.exists(path):
    os.makedirs(path)

    

In [ ]:
# summary of each law use LLM use Bert

from transformers import BertTokenizer, BertForSequenceClassification

# Load the pre-trained model and tokenizer
model = BertForSequenceClassification.from_pretrained("nlpaueb/legal-bert-base-uncased")

tokenizer = BertTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")




In [ ]:
# Process each law section

for i in range(1, 49):
    # Read the content of the law section
    with open(f"Laws/law_{i}.txt", "r") as file:
        content = file.read()
    # Tokenize the content
    inputs = tokenizer(content, return_tensors="pt", padding=True, truncation=True)
    # Perform the classification
    outputs = model(**inputs)
    # Get the predicted label
    predicted_label = outputs.logits.argmax().item()
    # Save the summary to a file
    with open(f"{path}summary_{i}.txt", "w") as file:
        file.write(f"Law {i}\n")
        file.write(f"Predicted label: {predicted_label}\n")
        file.write(content)
        file.write("\n\n")




In [ ]:
# Now we can use the summaries to search for specific laws based on the predicted label

# Define the search query
query = "contract"

# Process each summary file

for i in range(1, 49):

    # Read the content of the summary
    with open(f"Summaries/summary_{i}.txt", "r") as file:
        content = file.read()

    # Check if the query is present in the content
    if query in content:
        print(f"Law {i} contains the query '{query}'")



In [ ]:
# translate our laws to turkish use T5

from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the pre-trained model and tokenizer

model = T5ForConditionalGeneration.from_pretrained("t5-small")

tokenizer = T5Tokenizer.from_pretrained("t5-small")




In [ ]:
# Process each law section

for i in range(1, 49):
    # Read the content of the law section
    with open(f"Laws/law_{i}.txt", "r") as file:
        content = file.read()
    # Translate the content
    inputs = tokenizer(f"translate English to Turkish: {content}", return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(**inputs)
    translated_content = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Save the translation to a file
    with open(f"Translations/translation_{i}.txt", "w") as file:
        file.write(f"Law {i}\n")
        file.write(translated_content)
        file.write("\n\n")

    

In [ ]:
# read the translated laws and search for a specific term

# Define the search query
query = "sözleşme"

# Process each translation file
for i in range(1, 49):
    # Read the content of the translation
    with open(f"Translations/translation_{i}.txt", "r") as file:
        content = file.read()
    # Check if the query is present in the content
    if query in content:
        print(f"Law {i} contains the query '{query}'")

    